# **Predicting IMDB Ratings Based on Movie Reviews**

Authors: Yuanzhe Marco Ma, Arash Shamseddini, Kaicheng Tan, Zhenrui Yu

## Table of Contents

- [Project Overview](#1)
- [Data](#2)
- [Exploratory Data Analysis (EDA)](#3)
- [Prediction](#4)
    - [Preprocessing](#4)
    - [Model Fitting](#5)
    - [Results](#6)
- [Reflection](#7)
- [References](#8)

# I. Project Overview <a class="anchor" id="1"></a>
<hr>

In this project, we look at the relationship between movie reviews and their IMDB scores (ranging from 0 ~ 10). Positive reviews are often related to high IMDB scores, while negative reviews indicate the opposite. While it is easy for humans to understand a piece of review and guess the scores, we wonder if machines could understand it as well. Furthermore, we would like to automate this process, so that given a bunch of movie reviews, we are able to predict their corresponding IMDB scores easily. <br>

In this project, most of the data wrangling in this project is powered by the [pandas](#c4) (McKinney 2010) library. The machine learning models and frameworks utilize [sklearn](#c3) (Pedregosa et al. 2011). We use Support Vector Regressor (SVR) as our prediction model. <br>

# II. Where Do We Get the Data? <a class="anchor" id="2"></a>
<hr>

We obtained our data from an open-sourced github repository: <br> 
> https://github.com/nproellochs/SentimentDictionaries/blob/master/Dataset_IMDB.csv <br>

The repository was originally used for sentiment analysis related to movie reviews. Here we are using the `Dataset_IMDB.csv` as our main data source. 

#### Automation
To automate data retrieval, we have written a script to obtain the dataset with Python. The script can be accessed [here](../src/download_data.py). 

# III. What does the data look like? <a class="anchor" id="3"></a>
<hr>

Let's look into the dataset by performing some **Exploratory Data Analysis (EDA)**. <br>

#### 1. Data Columns:
| Column Name | Column Type | Description                             |
|-------------|-------------|-----------------------------------------|
| Id          | Numeric     | Unique ID assigned to each observation. |
| Text        | Free Text   | Body of the review content.             |
| Author      | Categorical | Author's name of the review             |
| Rating      | Numeric     | Ratings given along with the review (normalized)   |

For this project, we look primarily into the `Text` and `Rating` columns.  



We realized that `Author` may have significant relation to ratings, but since we are making a generalized model for reviews from any audience, we have decided to discard it for this analysis. Therefore, we will **drop** both the `Author` and `Id` columns.

#### 2. The `Text` feature
The `Text` feature contains all the movie reviews. This will be our primarily input feature. <br>
Below are the top 10 most frequent words in the reviews:

| Word   | Frequency | Rank |
|--------|-----------|------|
|the     |   172557  |  1   |
|of      |   78038   |  2   |
|and     |   76392   |  3   |
|to      |   74239   |  4   |
|is      |   57547   |  5   |
|in      |   49646   |  6   |
|that    |   33476   |  7   |
|it      |   33061   |  8   |
|as      |   27536   |  9   |
|with    |   26852   |  10  |

As we can see, the most frequent words are often generic words such as prepositions and pronouns, which has little implication to our model learning. We might want to avoid overfitting to these words as we train our model. 

#### 3. The `Rating` Class
`Ratings` will be our target class. Let's look at a distribution of `Rating`. 

![ratings_histo](../results/histogram_rating_distribution.svg)

The ratings seem roughly normally distributed, with a little skewness to the left. Most of the ratings cluster around 0.5 ~ 0.8. 

#### 4. Correlation between `Text` length and `Rating`
We suspect that people more passionate about certain movies tend to write longer reviews to express feelings. This could also be true for very negative reviews. <br> <br>
A bar plot of `Text` length vs `Rating` is presented below. 

![textlength_vs_rating](../results/histogram_rating_vs_text_length.svg)

There doesn't seem to be a strong correlation between reviews length and rating. However, it is notable that for the most positive ratings (from 0.7 ~ 1.0), the reviews tend to be higher. 

# IV. Pre-processing <a class="anchor" id="4"></a>
<hr>

## New Columns

In addition to `Text` feature alone, we extracted two potentially useful columns that could enhance our machine learning model. 

#### 1. `n_words`
As mentioned above, we suspect some correlation between review lengths and ratings. Therefore we created an `n_words` feature, which counts the number of words in each review.  
#### 2. `sentiment`
We utilized the [**NLTK**](#c2) (Bird 2009) package to assist us in extracting the sentiment of each review. This `sentiment` feature will have four ordinal categories - ['neg', 'compound', 'neu', 'pos']. 

## Column Transformer

Now we have three main columns to transform before fitting - `Review`, `n_words` and `sentiment`. 
- `Review`: since it is text feature, we will use sklearn's `CountVectorizer` to transform the text into bag of words. As mentioned in the EDA, since the most frequent words are often meaningless, we added `stop_words=20` argument into the vectorizer to exclude the top 20 most frequent words.
- `n_words`: we standardize the data with sklearn's `StandardScaler` to avoid its scaling effect on the models. 
- `sentiment`: since it is ordinal data, we utilize sklearn's `OrdinalEncoder` to encode the ordinal data in one-hot style. 

# V. Model Selection & Fitting <a class="anchor" id="5"></a>

Now we have our training data ready, we will consider which model to use to fit. <br>
We selected 3 candidate models that are suitable for regression prediction:
1. Support Vector Regressor (SVR)
2. Random Forest Regressor
3. Ridge Regressor

We tuned the hyperparameters of each model using sklearn's [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). <br>
Below are comparative cross-validation scores of the 3 models.

| model                 | fit_time   | score_time | test_score   |  train_score |
|-----------------------|------------|------------|--------------|--------------|
| Ridge                 | 2.198422   | 0.511951   | 0.532790     |  0.799778    |
| SVR                   | 28.988741  | 8.086322   | 0.469965     |  0.723018    |
| RandomForestRegressor | 287.661862 | 0.499031   | 0.359660     |  0.906179    |

### Winner: `SVR`

We selected `SVR` with a combindation of two considerations - cross-validation scores and the risk of overfitting. `SVR` was selected because it has a CV score comparable to the `Ridge` regressor, while having the least amount of overfitting. For more details on the model selection process, see [this report](../doc/model_selection.ipynb).

Below is a detailed specification of our final model. 

| Model Name | Hyperparameter - Gamma | Mean Fit Time |  Mean Scoring Time | Mean CV Score |
|------------|------------------------|---------------|--------------------|-------------|
|    SVR     |  0.0007 | 28.988741s | 8.086322s |0.469965 |

# VI. Prediction Results <a class="anchor" id="6"></a>

Below is our model's prediction scores on the test dataset.

In [11]:
import pandas as pd

scores = pd.read_csv('../results/model_test_scores.csv', index_col=0)
scores

,r2,rmse
model,0.50371,1.286077




#### 1. **$R^2$**, RMSE Scores

$R^2$ and RMSE (Root-mean-squared-error) are two common metrics for evaluating a regression model's accuracy. <br> 

The obtained $R^2$ score for the test set is **0.5037**, which was comparable to, and even better than our validation score. The RMSE score was **1.2861**. This seems large because it means our predicted score can have a margin of error of over 1.2 points. 

#### 2. Prediction vs. True Ratings
We have also created a scatterplot to compare our predicted ratings vs. the true ratings. 

![true_vs_predict](../results/true_vs_predict.svg)

> There is an obvious difference between the predicted ratings and true ratings. In the true ratings, people tend to give whole number rating, i.e. 0.3 instead of 0.3247. Our model did not capture that. 

Despite that, most of the points are somewhat clustered around the identity line. This indicates that our model didn't seem to under-fit or over-fit. 

# VII. Reflection and Improvements <a class="anchor" id="7"></a>

Now that we are done with our prediction and analysis, we can examine the quality of our work. 
In fact, there are a few areas of improvement. 

- We discarded the `Author` and `id` columns at the beginning. In fact, these columns may be influential features, especially `Author`. One major flaw of our dataset is that all reviews come from four critics. This makes it difficult to generalize to the broader audience. It would better if we can obtain reviews from the general audiences. 

- There is risk of overfitting in our models. Based on the model selection results, all three candidate models have large gaps between training score and test scores. We have tuned the hyperparameters the best we can, yet the gap is still apparent. This might have to do with the nature of our dataset and model compatibility, and it requires further analysis. 

- As mentioned above, our model did not capture the pattern where humans tend to give whole number scores. In the future, we can probably give more emphasis to predicting whole number scores, so that our model resembles more human behaviors. 

- The `sentiment` feature genereated with the NLTK  package contained only `neu` and `compound` in our dataset. This is confusing and we have yet to understand this behavior. We included the feature regardless because it may still provide useful information. However, this is definitely a place we need to investigate further into. 

## References <a class="anchor" id="8"></a>

1. Dua, Dheeru, and Casey Graff. 2017. “UCI Machine Learning Repository.” University of California, Irvine, School of Information; Computer Sciences. (http://archive.ics.uci.edu/ml) <a class="anchor" id="c1"></a>
2. Bird, Steven, Edward Loper and Ewan Klein (2009). Natural Language Processing with Python.  O'Reilly Media Inc. (https://www.nltk.org/) <a class="anchor" id="c2"></a>
3. Scikit-learn: Machine Learning in Python, Pedregosa et al., JMLR 12, pp. 2825-2830, 2011. (https://jmlr.csail.mit.edu/papers/v12/pedregosa11a.html) <a class="anchor" id="c3"></a>
4. McKinney, W., & others. (2010). Data structures for statistical computing in python. In Proceedings of the 9th Python in Science Conference (Vol. 445, pp. 51–56). (https://pandas.pydata.org/)  <a class="anchor" id="c4"></a>
4. Gareth James, Daniela Witten, Trevor Hastie and Rob Tibshirani, 2009, “An Introduction to Statistical Learning with Application in R”, Springer Publishing